In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pymc3 as pm
print('Running on PyMC3 v{}'.format(pm.__version__))

from bs4 import BeautifulSoup
import requests
import re

import os

from sklearn.externals import joblib 

import pandas as pd

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\linds\Anaconda3\envs\pymc3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC3 v3.8


C:\Users\linds\Anaconda3\envs\pymc3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
country = 'Canada'
file = os.path.join('csv_out',country+'.csv')
country_df = pd.read_csv(file)
country_df

def get_country_sir(country, start_date='', end_date='', min_cases=10):
    
    # update for any country
    # italy
    population = 60e6
    
    country = country.title().replace(' ', '_')
    file = os.path.join('csv_out',country+'.csv')
    country_df = pd.read_csv(file)
    
    start = country_df[country_df.Date == start_date].index
    if len(start)==0:
        start = 0
    else:
        start = start[0]
        
    end = country_df[country_df.Date == end_date].index
    if len(end)==0:
        end = country_df.index[-1]
    else:
        end = end[0]
        
    dates = country_df.loc[start:end + 1, 'Date'].values
    data = country_df.loc[start:end + 1, 'Confirmed'].values
    deaths = country_df.loc[start:end + 1, 'Deaths'].values
    recovered = country_df.loc[start:end+1, 'Recovered'].values

    if max(data) < min_cases:
        print('Warning, {:d} cases has not occured in this date range.')
    else:
        min_start = np.where(np.array(data) >= min_cases)[0][0]
        data = data[min_start:]
        dates = dates[min_start:]
        deaths = deaths[min_start:]
        recovered = recovered[min_start:]
        
    # infected = total cases - deaths - recoveries
    infected = data - deaths - recovered
    
    # susceptible = population - infected - deaths - recovered
    susceptible = population - infected - deaths - recovered
        
    

    return dates, np.arange(1, len(data) + 1), susceptible/population, infected/population
    
    

In [4]:
dates, x, sus, inf = get_country_sir('italy', min_cases=100)

In [ ]:
def sir_function(y, t, p):
    ds 